In [1]:

import torch
from torchvision.datasets import CelebA
from torchvision.transforms.v2 import (
    CenterCrop,
    ColorJitter,
    Compose,
    RandomHorizontalFlip,
    ToDtype,
    ToImage,
)

# Initial datatype of the images, will be converted by autocast later
DTYPE = torch.float32

## change the path to the path where the data is stored
# path = "C:/Users/modi/Desktop/ecse 552/Project/Facial-Attribute-Recognition-main/model/data"
path = "C:/Users/manju/Desktop/ECSE 552/Project"
train_transform = Compose(
    [
        # RandomRotation(15, interpolation=InterpolationMode.BILINEAR),
        CenterCrop(178),
        ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        RandomHorizontalFlip(),
        ToImage(),
        ToDtype(dtype=DTYPE, scale=True),
    ]
)

normal_transform = Compose([CenterCrop(178), ToImage(), ToDtype(DTYPE, scale=True)])

train = CelebA(path, "train",  transform=train_transform)
test = CelebA(path, "test", transform=normal_transform)
val = CelebA(path, "valid", transform=normal_transform)
import random
import textwrap

import matplotlib.pyplot as plt

ATTRIBUTES = train.attr_names[:-1]

print(f"Number of pictures: {len(train)}")

print(f"Number of attributes: {len(ATTRIBUTES)}")

print(f"Attribute names: {', '.join(ATTRIBUTES)}")

shape = train[0][0].shape
print(f"Resolution: {shape[1]}x{shape[2]}")


Number of pictures: 162770
Number of attributes: 40
Attribute names: 5_o_Clock_Shadow, Arched_Eyebrows, Attractive, Bags_Under_Eyes, Bald, Bangs, Big_Lips, Big_Nose, Black_Hair, Blond_Hair, Blurry, Brown_Hair, Bushy_Eyebrows, Chubby, Double_Chin, Eyeglasses, Goatee, Gray_Hair, Heavy_Makeup, High_Cheekbones, Male, Mouth_Slightly_Open, Mustache, Narrow_Eyes, No_Beard, Oval_Face, Pale_Skin, Pointy_Nose, Receding_Hairline, Rosy_Cheeks, Sideburns, Smiling, Straight_Hair, Wavy_Hair, Wearing_Earrings, Wearing_Hat, Wearing_Lipstick, Wearing_Necklace, Wearing_Necktie, Young
Resolution: 178x178


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [19]:

# --- CNN MODEL ---
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=2),  # [3, 178, 178] -> [8, 88, 88]
            nn.ReLU(),
            nn.MaxPool2d(2),                          # [8, 88, 88] -> [8, 44, 44]
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8 * 44 * 44, num_classes),
            nn.Sigmoid(),  # for multilabel
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x
# --- Transforming labels ---
def transform_batch_labels(batch):
    images, labels = batch
    return images.to(device), labels.float().to(device)  # ← no transform_y


# --- Setup ---
num_classes = 40
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes = 40).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

train_loader = DataLoader(train, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val, batch_size=64, shuffle=False, num_workers=2)

# --- Training Function ---
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for batch in tqdm(loader, desc="Training"):
        images, labels = transform_batch_labels(batch)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(loader)

# --- Validation Function ---
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            images, labels = transform_batch_labels(batch)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(loader)



In [20]:
import os
import torch

# Detect GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# Folder to save model
save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, "best_model.pt")

# --- Early Stopping Config ---
patience = 3
best_val_loss = float("inf")
epochs_without_improvement = 0
NUM_EPOCHS = 50

for epoch in range(NUM_EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), model_path)
        print(f"✅ Validation loss improved. Model saved to {model_path}")
    else:
        epochs_without_improvement += 1
        print(f"⏸️ No improvement for {epochs_without_improvement} epoch(s).")

        if epochs_without_improvement >= patience:
            print("🛑 Early stopping triggered.")
            break

# --- Load model with proper device map ---
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
print("🔁 Loaded best model weights and moved to GPU.")


In [21]:
import torch
import numpy as np

def evaluate_and_collect(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            images, labels = transform_batch_labels(batch)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            pred_binary = (outputs > 0.5).float()
            all_preds.append(pred_binary.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    avg_loss = running_loss / len(loader)
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    print(f"\n✅ Evaluation completed — Avg Loss: {avg_loss:.4f}")
    return all_preds, all_labels



In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

In [23]:
test_loader = DataLoader(test, batch_size=64, shuffle=False, num_workers=2)

In [24]:
all_preds, all_labels = evaluate_and_collect(model, test_loader, criterion = nn.BCELoss(), device = device)

In [25]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure preds and labels are numpy arrays
# all_preds: binary predictions (0/1), shape: [N, 40]
# all_labels: ground truth labels (0/1), shape: [N, 40]

# Element-wise correctness
correct_matrix = (all_preds == all_labels).astype(np.float32)

# Per-attribute accuracy
attribute_accuracy = correct_matrix.mean(axis=0)  # shape: [40]

# Overall accuracy (average across all attributes and samples)
overall_accuracy = correct_matrix.mean()

print(f"Overall accuracy: {overall_accuracy:.4f}")


In [26]:
correct_matrix = (all_preds == all_labels).astype(np.float32)
attribute_accuracy = correct_matrix.mean(axis=0)  # shape: (40,)

# Print each attribute's accuracy
for i, attr in enumerate(ATTRIBUTES):
    print(f"{attr:25s}: {attribute_accuracy[i]:.4f}")

In [27]:
import pandas as pd
import numpy as np

def save_preds_labels_to_csv(all_preds, all_labels, filename="results.csv", attributes=None):
    """
    Save predictions and labels to a CSV file side-by-side.
    If attribute names are given, use them as column headers.
    """
    # Make sure arrays are numpy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Prefix column names
    if attributes is None:
        attributes = [f"attr_{i}" for i in range(all_preds.shape[1])]

    pred_cols = [f"pred_{attr}" for attr in attributes]
    label_cols = [f"true_{attr}" for attr in attributes]

    df_preds = pd.DataFrame(all_preds, columns=pred_cols)
    df_labels = pd.DataFrame(all_labels, columns=label_cols)

    df = pd.concat([df_preds, df_labels], axis=1)
    df.to_csv(filename, index=False)
    print(f"✅ Saved predictions and labels to {filename}")


In [30]:
save_preds_labels_to_csv(all_preds, all_labels, filename="cnn_basic_results_1.csv", attributes=ATTRIBUTES)
